In [0]:
from datetime import datetime
import urllib.request
import zipfile

In [0]:
# Catálogo databricks camada bronze

catalogo =  "fiscaliza_legislativo"
schema = "bronze"
volume = "arquivos"
volumeTemp = "tmp"
diretorio = "despesas_deputados"
anoInicio = 2020
anoAtual = int(datetime.now().year)

PATH_VOLUME = f"/Volumes/{catalogo}/{schema}/{volume}/{diretorio}"
PATH_TEMP_VOLUME = f"/Volumes/{catalogo}/{schema}/{volumeTemp}"

# Inicia execução

In [0]:

TInicio = datetime.now()
print(TInicio)

## Criação de Esquema e Volume

In [0]:
# Cria Schema Bronze

SQL =  f"""
CREATE SCHEMA IF NOT EXISTS {catalogo}.{schema}
"""
spark.sql(SQL)

In [0]:
# Cria Volume arquivos

SQL =  f"""
CREATE VOLUME IF NOT EXISTS {catalogo}.{schema}.{volume}
"""
spark.sql(SQL)

In [0]:
# Cria diretorio despesas_deputados

dbutils.fs.mkdirs(PATH_VOLUME)

## Cria Volume tmp

In [0]:
# Cria Volume tmp  para salvar arquivos temporarios

SQL =  f"""
CREATE VOLUME IF NOT EXISTS {catalogo}.{schema}.{volumeTemp}
"""
spark.sql(SQL)

### Lista Arquivos do Volume

In [0]:
# Lista Arquivos da pasta despesas_deputados

info_arquivos = dbutils.fs.ls(PATH_VOLUME)
arquivos = [arquivo.name for arquivo in info_arquivos]

## Baixa Arquivos

In [0]:
# Baixa apenas arquivos que ainda não estão no volume,e atualiza o arquivo do ano atual

for ano in range(anoInicio, anoAtual+1):
  url = f"https://www.camara.leg.br/cotas/Ano-{ano}.xml.zip"
  path_destino = f"{PATH_TEMP_VOLUME}/{ano}.zip"

  if f"Ano-{ano}.xml" not in arquivos or ano == anoAtual:
    print(f"Baixando arquivo {ano}.zip")
    try:
       urllib.request.urlretrieve(url, path_destino)
       print(f"Download concluído com sucesso: {path_destino}")
    except Exception as e:
      print(f"Erro ao baixar arquivo {ano}.zip")
      print(e)
  

## Extrai Arquivos ZIP

In [0]:
# Lista Arquivos da pasta temp

info_arquivos_temp = dbutils.fs.ls(PATH_TEMP_VOLUME)
arquivos_temp = [arquivo.path.replace("dbfs:", "") for arquivo in info_arquivos_temp]

In [0]:
# Extrai todos os arquivos da pasta tmp no volume despesas_deputados

for arquivo_zip in arquivos_temp:
  with zipfile.ZipFile(arquivo_zip, 'r') as zip:
    zip.extractall(PATH_VOLUME)

## Remove Arquivos Temporarios

In [0]:

# Remove todos os arquivos da pasta tmp

dbutils.fs.rm(PATH_TEMP_VOLUME, recurse=True)

In [0]:
TFim = datetime.now()
print(TFim)
print("Duração:", TFim - TInicio)